In [1]:
import torch
from torch import nn
from entmax import entmax_bisect
from torch.autograd import grad
from tqdm import tqdm
from entmax.activations import sparsemax, entmax15

In [2]:
#alpha = nn.Parameter(torch.Tensor([1.5]))
#alpha

In [3]:
class AlphaChooser(torch.nn.Module):

    def __init__(self, head_count):
        """head_count (int): number of attention heads"""
        super(AlphaChooser, self).__init__()
        self.pre_alpha = nn.Parameter(torch.randn(head_count))

    def forward(self):
        alpha = 1 + torch.sigmoid(self.pre_alpha)
        return torch.clamp(alpha, min=1.01, max=2)

In [4]:
att_scores = torch.rand(128, 12, 36, 1024)

In [5]:
alpha = AlphaChooser(1)

In [6]:
alpha()

tensor([1.4383], grad_fn=<ClampBackward>)

In [7]:
p = 0.
for i in range(att_scores.size(1)):
    p += entmax_bisect(X = att_scores,alpha = alpha()[i], dim=1)

In [8]:
p = p/att_scores.size(1)

In [9]:
p.shape

torch.Size([128, 12, 36, 1024])

In [11]:
p = entmax_bisect(att_scores,alpha()[0])

In [23]:
p.mean()

tensor(0.0010, grad_fn=<MeanBackward0>)

In [22]:
p.mean().backward()

In [18]:
t = torch.Tensor([[0.0460, 0.3276, 0.6264],
        [0.0026, 0.1012, 0.8963]])